In [4]:
import tensorflow as tf
import numpy as np
import threading

Load data

In [20]:
data = np.genfromtxt('features.txt')


In [95]:
all_input = data[:,0:25]
all_output = .5*data[:,25:27]+.5
train_input = data[:1000000,0:25]
train_output = .5*data[:1000000,25:27]+.5
test_input = data[1000000:,0:25]
test_output = .5*data[1000000:,25:27]+.5
#Don't do this!
train_input = all_input
train_output = all_output

In [44]:
train_input = data[356048:,0:25]
train_output = .5*data[356048:,25:27]+.5
test_input = data[:356048,0:25]
test_output = .5*data[:356048,25:27]+.5

Randomize frames so we can do mini-batches. Don't do that for RNN

In [97]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:]
    shuffled_labels = labels[permutation,:]
    return shuffled_dataset, shuffled_labels
train_input, train_output = randomize(train_input, train_output)
test_input, test_output = randomize(test_input, test_output)

In [41]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:]
    shuffled_labels = labels[permutation,:]
    return shuffled_dataset, shuffled_labels
all_input, all_output = randomize(all_input, all_output)
train_input = all_input[:1000000,:]
train_output = all_output[:1000000,:]
test_input = all_input[1000000:,:]
test_output = all_output[1000000:,:]


Setup MLP definition

In [156]:
#batch_size = 1356048
batch_size = 13560
hidden_size = 16
feature_size = 25
nb_outputs = 2

def my_cross_entropy(x,z):
    entropy = x - x * z + tf.log(1 + tf.exp(-x))
    #entropy = entropy * (z!=0.5)
    entropy = entropy * (2*tf.abs(z-0.5))
    entropy = entropy * [5, 1]
    return entropy

def my_mse(x,z):
    mse = (x - z)**2
    mse = mse * (2*np.abs(z-0.5))
    rms = 5*mse[:,0] + mse[:,1]
    return 2*np.sqrt(np.mean(rms))


graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, feature_size))
    tf_train_output = tf.placeholder(tf.float32, shape=(None, nb_outputs))
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([feature_size, hidden_size]))
    biases = tf.Variable(tf.zeros([hidden_size]))

    logits1 = tf.matmul(tf_train_dataset, weights) + biases
    hidden = tf.nn.tanh(logits1)

    weights2 = tf.Variable(
        tf.truncated_normal([hidden_size, nb_outputs]))
    biases2 = tf.Variable(tf.zeros([nb_outputs]))
    
    # Training computation.
    logits = tf.matmul(hidden, weights2) + biases2
    loss = tf.reduce_mean(
        my_cross_entropy(logits, tf_train_output))
  
    # Optimizer.
    optimizer = tf.train.AdamOptimizer().minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.sigmoid(logits)
    tf_test_dataset = tf.to_float(tf.constant(test_input))
    test_prediction = tf.nn.sigmoid(tf.matmul(tf.nn.tanh(tf.matmul(tf_test_dataset, weights) + biases), weights2) + biases2)


In [157]:
num_steps = 1000001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_output.shape[0] - batch_size)
        #offset = 0
        # Generate a minibatch.
        batch_data = train_input[offset:(offset + batch_size), :]
        batch_output = train_output[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_output : batch_output}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 10000 == 0):
            W0 = weights.eval()
            B0 = biases.eval()
            W1 = weights2.eval()
            B1 = biases2.eval()
            #print("Minibatch loss at step %d: %f" % (step, l))
            pred = train_prediction.eval(feed_dict = {tf_train_dataset: train_input, tf_train_output: train_output})
            tpred = test_prediction.eval()
            #print(np.mean(abs(pred - train_output)>0.5000, axis=0))
            #print(my_mse(pred, train_output))
            print("step %d:" % step, np.mean(abs(pred - train_output)>0.5000, axis=0), my_mse(pred, train_output))

Initialized
('step 0:', array([ 0.33901381,  0.6559989 ]), 2.7593063301604341)
('step 10000:', array([ 0.01055051,  0.03929876]), 0.5309413611569257)
('step 20000:', array([ 0.00821284,  0.03636818]), 0.48530305543486219)
('step 30000:', array([ 0.00771875,  0.03502162]), 0.47348787198022779)
('step 40000:', array([ 0.00746581,  0.03458727]), 0.46810315897765681)
('step 50000:', array([ 0.00735888,  0.03429008]), 0.46477801969079563)
('step 60000:', array([ 0.00720402,  0.03418979]), 0.46339541788187744)
('step 70000:', array([ 0.00712143,  0.03397151]), 0.46108325315043203)
('step 80000:', array([ 0.00685595,  0.03337419]), 0.45583643011508423)
('step 90000:', array([ 0.00690389,  0.03287715]), 0.45430868711897104)
('step 100000:', array([ 0.00677705,  0.03293762]), 0.45172379367593218)
('step 110000:', array([ 0.00681171,  0.03297007]), 0.4526366497010374)
('step 120000:', array([ 0.00664873,  0.03267141]), 0.44946324262374254)
('step 130000:', array([ 0.00664726,  0.03244723]), 0.44

KeyboardInterrupt: 

In [134]:
WW0 = np.concatenate((W0, np.reshape(B0, (1, 16))), axis=0)
np.reshape(np.transpose(WW0), (1, 26*16))

array([[ -3.28951739e-02,   1.65119004e-02,   1.01238862e-02,
          8.55314210e-02,   6.22863695e-02,   5.94221503e-02,
          2.54944582e-02,  -8.97842348e-02,   2.48227324e-02,
         -4.25750315e-02,   1.41070848e-02,   1.01724721e-01,
          1.80129543e-01,   2.57834122e-02,  -3.53035986e-01,
          7.45099261e-02,  -2.10935637e-01,  -3.21772397e-01,
         -8.96112502e-01,  -2.04263225e-01,   1.73088157e+00,
         -8.18579197e-01,   1.75526047e+00,   1.49274802e+00,
          1.63044304e-01,  -8.01915348e-01,   4.12034504e-02,
          1.17459960e-01,   9.67575517e-03,   3.42897512e-02,
          1.91381034e-02,  -1.21345274e-01,   6.57919347e-02,
         -1.53688330e-03,  -4.90540527e-02,   2.74076443e-02,
         -9.34747681e-02,  -9.96665880e-02,  -4.31297302e-01,
         -7.20995246e-03,  -9.64779079e-01,  -2.02184841e-02,
          2.81997681e-01,   1.10685110e-01,  -3.71672779e-01,
          1.36981398e-01,  -3.80686831e+00,  -6.70365095e-01,
        